## 1. Setup Environment

In [ ]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

!pip install -q tensorboardX
!git clone https://github.com/ShuweiShao/AF-SfMLearner
repo_path = "/content/AF-SfMLearner"


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.3 MB/s eta 0:00:00
Cloning into 'AF-SfMLearner'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 254 (delta 40), reused 60 (delta 32), pack-reused 183 (from 1)
Receiving objects: 100% (254/254), 4.76 MiB | 34.31 MiB/s, done.
Resolving deltas: 100% (114/114), done.
✅ Environment setup hoàn tất!


Data prepare

In [ ]:
import os
import glob
import zipfile
from tqdm import tqdm

train_zip_path = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBufferS2S8.zip"
training_data_root = "/content/training_data_simcol"

if not os.path.exists(training_data_root):
    print(f" Extracting: {train_zip_path}...")
    os.makedirs(training_data_root, exist_ok=True)

    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(training_data_root)

    # Get sequences lists
    subfolders = [f.path for f in os.scandir(training_data_root) if f.is_dir()]
    print(f"Sequences: {[os.path.basename(f) for f in subfolders]}")

    # Correct file name: FrameBuffer_0000.png -> 0000000000.png
    print("Correct file name")
    for folder in subfolders:
        images = glob.glob(os.path.join(folder, "*.png"))
        for img_path in tqdm(images, desc=f"Rename {os.path.basename(folder)}"):
            folder_path, filename = os.path.split(img_path)
            if "FrameBuffer_" in filename:
                new_name = filename.replace("FrameBuffer_", "")
                try:
                    num = int(new_name.split('.')[0])
                    new_filename = "{:010d}.png".format(num)
                    os.rename(img_path, os.path.join(folder_path, new_filename))
                except ValueError:
                    pass

    print(f"Data ready at: {training_data_root}")
else:
    subfolders = [f.path for f in os.scandir(training_data_root) if f.is_dir()]
    print(f"Data has existed: {training_data_root}")
    print(f"Sequences: {[os.path.basename(f) for f in subfolders]}")

⏳ Đang giải nén dữ liệu từ /content/drive/MyDrive/3D_reconstruction/datasets/FrameBufferS2S8.zip...
📂 Sequences: ['FrameBuffer_S5', 'FrameBuffer_S8', 'FrameBuffer_S6', 'FrameBuffer_S4', 'FrameBuffer_S7', 'FrameBuffer_S2', 'FrameBuffer_S3']
⏳ Đang chuẩn hóa tên file...


Rename FrameBuffer_S3: 100%|██████████| 1201/1201 [00:00<00:00, 49259.83it/s]

✅ Dữ liệu sẵn sàng tại: /content/training_data_simcol


Split Files

In [ ]:
import os
import glob

repo_path = "/content/AF-SfMLearner"
training_data_root = "/content/training_data_simcol"
split_name = "simcol_sequences"
split_dir = os.path.join(repo_path, "splits", split_name)

os.makedirs(split_dir, exist_ok=True)

sequence_folders = sorted([f.name for f in os.scandir(training_data_root) if f.is_dir()])
print(f" Create split for sequences: {sequence_folders}")

all_train_lines = []
all_val_lines = []

for seq_name in sequence_folders:
    seq_path = os.path.join(training_data_root, seq_name)
    images = glob.glob(os.path.join(seq_path, "*.png"))
    indices = sorted([int(os.path.basename(img).split('.')[0]) for img in images])

    if len(indices) < 3:
        continue

    # Get rid of first and last frame
    valid_indices = indices[1:-1]

    # 90% train, 10% val
    split_idx = int(len(valid_indices) * 0.9)
    train_indices = valid_indices[:split_idx]
    val_indices = valid_indices[split_idx:]

    # Format: "folder  frame_index  side"
    for idx in train_indices:
        all_train_lines.append(f"{seq_name} {idx} l\n")
    for idx in val_indices:
        all_val_lines.append(f"{seq_name} {idx} l\n")

    print(f"   📁 {seq_name}: {len(train_indices)} train, {len(val_indices)} val")

# Ghi files
with open(os.path.join(split_dir, "train_files.txt"), "w") as f:
    f.writelines(all_train_lines)
with open(os.path.join(split_dir, "val_files.txt"), "w") as f:
    f.writelines(all_val_lines)

print(f"\n Split files at: {split_dir}")
print(f" Train: {len(all_train_lines)} samples")
print(f" Val: {len(all_val_lines)} samples")

🔄 Tạo split từ sequences: ['FrameBuffer_S2', 'FrameBuffer_S3', 'FrameBuffer_S4', 'FrameBuffer_S5', 'FrameBuffer_S6', 'FrameBuffer_S7', 'FrameBuffer_S8']
   📁 FrameBuffer_S2: 1079 train, 120 val
   📁 FrameBuffer_S3: 1079 train, 120 val
   📁 FrameBuffer_S4: 1079 train, 120 val
   📁 FrameBuffer_S5: 1079 train, 120 val
   📁 FrameBuffer_S6: 1079 train, 120 val
   📁 FrameBuffer_S7: 1079 train, 120 val
   📁 FrameBuffer_S8: 1079 train, 120 val

✅ Split files tại: /content/AF-SfMLearner/splits/simcol_sequences
📊 Tổng Train: 7553 samples
📊 Tổng Val: 840 samples


Patch

In [ ]:
import os
import numpy as np

repo_path = "/content/AF-SfMLearner"

def overwrite_file(path, content):
    with open(path, "w") as f:
        f.write(content)
    print(f"Write file: {os.path.basename(path)}")

def replace_in_file(path, old, new):
    if os.path.exists(path):
        with open(path, "r") as f:
            content = f.read()
        if old in content and new not in content:
            with open(path, "w") as f:
                f.write(content.replace(old, new))
            print(f"Patch: {os.path.basename(path)} - replace '{old[:30]}...'")
            return True
    return False

# Create SimColDataset

simcol_dataset_content = '''from __future__ import absolute_import, division, print_function
import os
import numpy as np
import PIL.Image as pil
from .mono_dataset import MonoDataset

class SimColDataset(MonoDataset):
    """SimCol3D Dataset - Load theo sequences
    Format file: folder  frame_index  side
    """
    def __init__(self, *args, **kwargs):
        super(SimColDataset, self).__init__(*args, **kwargs)

        # SimCol camera intrinsics (normalized)
        H, W = 475, 475
        fx_norm = 227.60416 / W
        fy_norm = 237.5 / H
        cx_norm = 227.604416 / W
        cy_norm = 237.5 / H

        self.K = np.array([[fx_norm, 0., cx_norm, 0.],
                           [0., fy_norm, cy_norm, 0.],
                           [0., 0., 1., 0.],
                           [0., 0., 0., 1.]], dtype=np.float32)

    def check_depth(self):
        return False

    def get_color(self, folder, frame_index, side, do_flip):
        # folder = sequence name (e.g., "FrameBuffer_S2")
        # frame_index = frame number within that sequence
        f_str = "{:010d}.png".format(frame_index)
        image_path = os.path.join(self.data_path, folder, f_str)

        with open(image_path, 'rb') as f:
            with pil.open(f) as img:
                color = img.convert('RGB')

        if do_flip:
            color = color.transpose(pil.FLIP_LEFT_RIGHT)
        return color
'''

overwrite_file(os.path.join(repo_path, "datasets", "simcol_dataset.py"), simcol_dataset_content)

# Fix ResnetEncoder

resnet_content = '''from __future__ import absolute_import, division, print_function
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetMultiImageInput(models.ResNet):
    def __init__(self, block, layers, num_input_images=1):
        super(ResNetMultiImageInput, self).__init__(block, layers)
        self.inplanes = 64
        self.conv1 = nn.Conv2d(num_input_images * 3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

def resnet_multiimage_input(num_layers, pretrained=False, num_input_images=1):
    blocks = {18: [2, 2, 2, 2], 50: [3, 4, 6, 3]}[num_layers]
    block_type = {18: models.resnet.BasicBlock, 50: models.resnet.Bottleneck}[num_layers]
    model = ResNetMultiImageInput(block_type, blocks, num_input_images=num_input_images)
    if pretrained:
        loaded = models.resnet18(weights="IMAGENET1K_V1").state_dict() if num_layers == 18 else models.resnet50(weights="IMAGENET1K_V1").state_dict()
        loaded['conv1.weight'] = torch.cat([loaded['conv1.weight']] * num_input_images, 1) / num_input_images
        model.load_state_dict(loaded)
    return model

class ResnetEncoder(nn.Module):
    def __init__(self, num_layers, pretrained, num_input_images=1):
        super(ResnetEncoder, self).__init__()
        self.num_ch_enc = np.array([64, 64, 128, 256, 512])
        if num_input_images > 1:
            self.encoder = resnet_multiimage_input(num_layers, pretrained, num_input_images)
        else:
            weights = "IMAGENET1K_V1" if pretrained else None
            resnets = {18: models.resnet18, 34: models.resnet34, 50: models.resnet50}
            self.encoder = resnets[num_layers](weights=weights)
        if num_layers > 34:
            self.num_ch_enc[1:] *= 4

    def forward(self, input_image):
        self.features = []
        x = self.encoder.conv1(input_image)
        x = self.encoder.bn1(x)
        self.features.append(self.encoder.relu(x))
        self.features.append(self.encoder.layer1(self.encoder.maxpool(self.features[-1])))
        self.features.append(self.encoder.layer2(self.features[-1]))
        self.features.append(self.encoder.layer3(self.features[-1]))
        self.features.append(self.encoder.layer4(self.features[-1]))
        return self.features
'''

overwrite_file(os.path.join(repo_path, "networks", "resnet_encoder.py"), resnet_content)

# Update __init__.py

init_path = os.path.join(repo_path, "datasets", "__init__.py")
with open(init_path, "r") as f:
    content = f.read()
if "SimColDataset" not in content:
    with open(init_path, "a") as f:
        f.write("\nfrom .simcol_dataset import SimColDataset\n")
    print(" Create SimColDataset in __init__.py")

# Update trainer_end_to_end.py - Add simcol into datasets_dict

trainer_path = os.path.join(repo_path, "trainer_end_to_end.py")
replace_in_file(trainer_path,
    'datasets_dict = {"endovis": datasets.SCAREDRAWDataset}',
    'datasets_dict = {"endovis": datasets.SCAREDRAWDataset, "simcol": datasets.SimColDataset}')

# Update options.py - Add choices for split and dataset

options_path = os.path.join(repo_path, "options.py")

# Add simcol_sequences to split choices
replace_in_file(options_path,
    'choices=["endovis", "eigen_zhou", "eigen_full", "odom", "benchmark"]',
    'choices=["endovis", "eigen_zhou", "eigen_full", "odom", "benchmark", "simcol_sequences"]')

# Add simcol to dataset choices
replace_in_file(options_path,
    'choices=["endovis", "kitti", "kitti_odom", "kitti_depth", "kitti_test"]',
    'choices=["endovis", "kitti", "kitti_odom", "kitti_depth", "kitti_test", "simcol"]')


# Fix deprecated functions in mono_dataset.py

mono_path = os.path.join(repo_path, "datasets", "mono_dataset.py")
replace_in_file(mono_path,
    "color_aug = transforms.ColorJitter.get_params(\n                self.brightness, self.contrast, self.saturation, self.hue)",
    "color_aug = transforms.ColorJitter(self.brightness, self.contrast, self.saturation, self.hue)")
replace_in_file(mono_path, "Image.ANTIALIAS", "Image.LANCZOS")

print("\n Finished patch!")

✅ Ghi file: simcol_dataset.py
✅ Ghi file: resnet_encoder.py
✅ Thêm SimColDataset vào __init__.py
✅ Patch: trainer_end_to_end.py - thay 'datasets_dict = {"endovis": da...'
✅ Patch: options.py - thay 'choices=["endovis", "eigen_zho...'
✅ Patch: options.py - thay 'choices=["endovis", "kitti", "...'
✅ Patch: mono_dataset.py - thay 'color_aug = transforms.ColorJi...'
✅ Patch: mono_dataset.py - thay 'Image.ANTIALIAS...'

🎉 Toàn bộ patches đã hoàn tất!


## 5. Training

In [ ]:
import sys
import os

repo_path = "/content/AF-SfMLearner"
if repo_path not in sys.path:
    sys.path.append(repo_path)

model_name = "simcol_sequences_model_320"
log_dir = "/content/drive/MyDrive/3D_reconstruction/outputs/AF-SfMLearner"
data_path = "/content/training_data_simcol"

os.makedirs(log_dir, exist_ok=True)

print(f" Training: {model_name}")
print(f" Data: Sequences  ")
print()

!cd "{repo_path}" && python train_end_to_end.py \
  --data_path "{data_path}" \
  --model_name "{model_name}" \
  --split "simcol_sequences" \
  --dataset "simcol" \
  --batch_size 8 \
  --learning_rate 1e-4 \
  --num_epochs 25 \
  --height 320 \
  --scheduler_step_size 10 \
  --width 320 \
  --log_dir "{log_dir}" \
  --frame_ids 0 -1 1 \
  --min_depth 1 \
  --max_depth 25 \
  --num_workers 2

🚀 Training: simcol_sequences_model_320
📊 Data: Sequences (không gộp)
📊 Epochs: 30
📊 Resolution: 320x320

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 164MB/s]
Training model named:
   simcol_sequences_model_320
Models and tensorboard events files are saved to:
   /content/drive/MyDrive/3D_reconstruction/outputs/AF-SfMLearner
Training is using:
   cuda
/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Using split:
   simcol_sequences
There are 7553 training items and 840 validation items

Training
/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:5100: UserWarning: Default grid_sample and affin

## 6. Inference (Test S1)

In [ ]:
import os
import shutil
import glob
import zipfile
from tqdm import tqdm

test_zip_path = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip"
test_extract_path = "/content/dataset_simcol_test"
test_data_root = "/content/test_data_simcol"
test_sequence_name = "simcol_S1_test"
test_target_dir = os.path.join(test_data_root, test_sequence_name)

if not os.path.exists(test_target_dir):
    os.makedirs(test_extract_path, exist_ok=True)
    with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
        zip_ref.extractall(test_extract_path)

    source_imgs_dir = None
    extensions = ['*.png', '*.jpg', '*.jpeg']
    for root, dirs, files in os.walk(test_extract_path):
        if any(glob.glob(os.path.join(root, ext)) for ext in extensions):
            source_imgs_dir = root
            break

    if not source_imgs_dir:
        raise FileNotFoundError("No data found!")

    print(f"Test frames at: {source_imgs_dir}")

    # Format file name
    print(" Correct file test name")
    os.makedirs(test_target_dir, exist_ok=True)
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(source_imgs_dir, ext)))
    image_files = sorted(image_files)

    for idx, file_path in enumerate(tqdm(image_files, desc="Formatting Test")):
        shutil.copy(file_path, os.path.join(test_target_dir, "{:010d}.png".format(idx)))

    print(f"\n Test data done: {test_target_dir}")
    print(f"Frames: {len(image_files)}")
else:
    print("\n Test data has exsited.")
    test_files = sorted(glob.glob(os.path.join(test_target_dir, "*.png")))
    print(f"Frames: {len(test_files)}")

⏳ Đang giải nén TEST DATA (S1) từ /content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip...
📂 Tìm thấy ảnh tại: /content/dataset_simcol_test/FrameBuffer_RGB
⏳ Đang chuẩn hóa tên file test...


Formatting Test: 100%|██████████| 1201/1201 [00:00<00:00, 1431.61it/s]

✅ Test data (S1) sẵn sàng tại /content/test_data_simcol/simcol_S1_test
📊 Tổng số frame: 1201


In [ ]:
# INFERENCE
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import glob
import os
import networks
from layers import transformation_from_parameters
import shutil

# --- CONFIG ---
model_name_folder = "simcol_sequences_model_320"
log_dir = "/content/drive/MyDrive/3D_reconstruction/outputs/AF-SfMLearner"

best_epoch = 24
final_output_dir = os.path.join(log_dir, model_name_folder, "models", f"weights_20")

test_target_dir = "/content/test_data_simcol/simcol_S1_test"

# Output
inference_out = "/content/inference_output_S1_test"
os.makedirs(inference_out, exist_ok=True)
device = torch.device("cuda")

infer_height = 320
infer_width = 320

print(f"Load Model: {final_output_dir}")
print(f"Test Data: {test_target_dir}")

def load_weights_safe(model, path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Không tìm thấy file: {path}")
    loaded_dict = torch.load(path, map_location=device)
    model_dict = model.state_dict()
    filtered_dict = {k: v for k, v in loaded_dict.items() if k in model_dict}
    model.load_state_dict(filtered_dict)
    return model

# --- LOAD MODELS ---
print("Loading Encoder")
enc = networks.ResnetEncoder(18, False).to(device)
load_weights_safe(enc, os.path.join(final_output_dir, "encoder.pth"))

print("Loading Depth Decoder")
dec = networks.DepthDecoder(enc.num_ch_enc).to(device)
load_weights_safe(dec, os.path.join(final_output_dir, "depth.pth"))

print("Loading Pose Encoder")
pose_enc = networks.ResnetEncoder(18, False, num_input_images=2).to(device)
load_weights_safe(pose_enc, os.path.join(final_output_dir, "pose_encoder.pth"))

print("Loading Pose Decoder")
pose_dec = networks.PoseDecoder(pose_enc.num_ch_enc, num_input_features=1, num_frames_to_predict_for=2).to(device)
load_weights_safe(pose_dec, os.path.join(final_output_dir, "pose.pth"))

for m in [enc, dec, pose_enc, pose_dec]:
    m.eval()

# --- INFERENCE ---
img_files = sorted(glob.glob(os.path.join(test_target_dir, "*.png")))
poses = [np.eye(4)]
current_pose = np.eye(4)
depth_dir = os.path.join(inference_out, "depth")
os.makedirs(depth_dir, exist_ok=True)

with torch.no_grad():
    for i in tqdm(range(len(img_files))):
        img_t_pil = Image.open(img_files[i]).convert('RGB').resize((infer_width, infer_height), Image.LANCZOS)
        img_t = torch.from_numpy(np.array(img_t_pil)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0

        # Predict Depth
        disp = dec(enc(img_t))[("disp", 0)]
        np.save(os.path.join(depth_dir, f"{i:010d}.npy"), disp.cpu().numpy())
        plt.imsave(os.path.join(depth_dir, f"{i:010d}.png"), disp.squeeze().cpu().numpy(), cmap='magma')

        # Predict Pose
        if i < len(img_files) - 1:
            img_next_pil = Image.open(img_files[i+1]).convert('RGB').resize((infer_width, infer_height), Image.LANCZOS)
            img_next = torch.from_numpy(np.array(img_next_pil)).permute(2,0,1).float().unsqueeze(0).to(device) / 255.0

            input_cat = torch.cat([img_next, img_t], 1)
            features = [pose_enc(input_cat)]
            axisangle, translation = pose_dec(features)

            pose_mat = transformation_from_parameters(axisangle[:, 0], translation[:, 0]).cpu().numpy()[0]
            current_pose = current_pose @ pose_mat
            poses.append(current_pose)

# --- SAVE TRAJECTORY ---
out_path = os.path.join(inference_out, "trajectory_S1.txt")
with open(out_path, "w") as f:
    for p in poses:
        row_str = " ".join(map(str, p[:3, :].flatten()))
        f.write(row_str + "\n")

print(f" Save trajectory at: {out_path}")

# Zip results
zip_name = "simcol_S1_test_results"
shutil.make_archive(os.path.join(log_dir, zip_name), 'zip', inference_out)
print(f" Save test result: {os.path.join(log_dir, zip_name + '.zip')}")

🚀 Load Model từ: /content/drive/MyDrive/3D_reconstruction/outputs/AF-SfMLearner/simcol_sequences_model_320/models/weights_20
📁 Test Data: /content/test_data_simcol/simcol_S1_test
--> Loading Encoder...
--> Loading Depth Decoder...
--> Loading Pose Encoder...
--> Loading Pose Decoder...
⏳ Đang xử lý 1201 frames từ TEST SET (S1)...


100%|██████████| 1201/1201 [01:48<00:00, 11.12it/s]


✅ Đã lưu trajectory tại: /content/inference_output_S1_test/trajectory_S1.txt
🎉 Kết quả TEST đã được lưu vào Drive: /content/drive/MyDrive/3D_reconstruction/outputs/AF-SfMLearner/simcol_S1_test_results.zip
